<a href="https://colab.research.google.com/github/tovantoi/CN-DA21TTA-TOVANTOI-Clean-Architecture/blob/main/IDM_VTON_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub==0.24.1 diffusers==0.25.0 transformers==4.35.2 accelerate==0.26.1 gradio==4.24.0 onnxruntime==1.16.2 einops==0.7.0 peft==0.5.0

  Using cached accelerate-0.26.1-py3-none-any.whl.metadata (18 kB)
  Using cached peft-0.5.0-py3-none-any.whl.metadata (22 kB)
Using cached accelerate-0.26.1-py3-none-any.whl (270 kB)
Using cached peft-0.5.0-py3-none-any.whl (85 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0
  Attempting uninstall: peft
    Found existing installation: peft 0.10.0
    Uninstalling peft-0.10.0:
      Successfully uninstalled peft-0.10.0


In [2]:
%cd /content
!git clone -b dev https://github.com/camenduru/IDM-VTON-hf

/content
fatal: destination path 'IDM-VTON-hf' already exists and is not an empty directory.


In [3]:
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/IDM-VTON/resolve/main/densepose/model_final_162be9.pkl -d /content/IDM-VTON-hf/ckpt/densepose

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/IDM-VTON/resolve/main/humanparsing/parsing_atr.onnx -d /content/IDM-VTON-hf/ckpt/humanparsing

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/IDM-VTON/resolve/main/humanparsing/parsing_lip.onnx -d /content/IDM-VTON-hf/ckpt/humanparsing

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/IDM-VTON/resolve/main/openpose/ckpts/body_pose_model.pth -d /content/IDM-VTON-hf/ckpt/openpose

aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
835f16|OK  |       0B/s|/content/IDM-VTON-hf/ckpt/densepose/b8a7382001b16e453bad95ca9dbc68ae8f2b839b304cf90eaf5c27fbdb4dae91

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
480343|OK  |       0B/s|/content/IDM-VTON-hf/ckpt/humanparsing/04c7d1d070d0e0ae943d86b18cb5aaaea9e278d97462e9cfb270cbbe4cd977f4

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
ae0319|OK  |       0B/s|/content/IDM-VTON-hf/ckpt/humanparsing/8436e1dae96e2601c373d1ace29c8f0978b16357d9038c17a8ba756cca376dbc

Status Legend:
(OK):download

In [ ]:
%cd /content/IDM-VTON-hf
!python app.py

/content/IDM-VTON-hf
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2025-05-06 07:03:28.931679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746515008.969681    8166 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746515008.980631    8166 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torc

In [ ]:
from flask import Flask, request, jsonify, send_from_directory
import os
from PIL import Image
from io import BytesIO
from datetime import datetime
from src.tryon_pipeline import StableDiffusionXLInpaintPipeline as TryonPipeline

app = Flask(__name__)
OUTPUT_DIR = "static/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Tải mô hình 1 lần khi khởi động API
pipe = TryonPipeline.from_pretrained(
    "weizmannscience/IDM-VTON",
    torch_dtype="float16",
    variant="fp16"
).to("cuda")

@app.route("/api/tryon", methods=["POST"])
def tryon():
    if "person" not in request.files or "cloth" not in request.files:
        return jsonify({"error": "Missing person or cloth image."}), 400

    person_file = request.files["person"]
    cloth_file = request.files["cloth"]
    prompt = request.form.get("prompt", "T-shirt")

    # Mở ảnh
    person_img = Image.open(BytesIO(person_file.read())).convert("RGB")
    cloth_img = Image.open(BytesIO(cloth_file.read())).convert("RGB")

    # Chạy mô hình thử đồ
    output = pipe(
        image=person_img,
        garment_image=cloth_img,
        garment_category=prompt
    ).images[0]

    # Lưu ảnh kết quả
    filename = f"tryon_{datetime.now().strftime('%Y%m%d%H%M%S')}.png"
    output_path = os.path.join(OUTPUT_DIR, filename)
    output.save(output_path)

    return jsonify({
        "message": "Try-on successful.",
        "result_url": f"/static/output/{filename}"
    })

@app.route("/static/output/<filename>")
def serve_result(filename):
    return send_from_directory(OUTPUT_DIR, filename)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=7860)


In [ ]:
!python /content/IDM-VTON-hf/api_flask.py